<h2>Demo av folium/leaflet</h2>

Aktiver cellene under, og start et kartsøk til slutt.

In [1]:
import pgeocode as pgeo

In [2]:
from collections import Counter
import folium
from folium.plugins import MarkerCluster
import dhlab.nbtext as nb
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
import qgrid as qg
import dhlab.token_map as tm
import ipywidgets as widgets
%matplotlib inline

In [4]:
from opencage.geocoder import OpenCageGeocode

# apinøkkel for opencage (men data er lastet ned, så den vil stort sett ikke benyttes)

opencage_api = "a8e2de88b5b64323a9b7668a0ee6e7b6"
geocoder = OpenCageGeocode(opencage_api)

def place_info(place, geo = geocoder):
    query = place  
    results = geo.geocode(query)
    return results

def koord(sted):
    coords = place_info(sted)[0]['geometry']
    return coords['lat'], coords['lng']

# kode for det norske kartverket sine data

def finnsted(sted):
    r = requests.get("https://ws.geonorge.no/SKWS3Index/ssr/sok", params = {'navn': sted})
    return r.text

    
# generer folium kart fra steder, et sted er en dict på formen {navn: [lat, long], navn2: [lat2, long2], ....}


def geokart(steder, limit=50):
    """ dict of places: {place: [lat, lng], ...} """
    import numpy as np
    
    #lat, lon = koord(sted)
    kartsnitt = folium.Map(location=[60, 10], zoom_start = 6)
    locs = []
    pops = []
    geo_tags = list(steder.values())
    geo_name = list(steder.keys())
    #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
    kartsnitt.add_child(MarkerCluster(geo_tags, popups = geo_name))
    #print(locs)
    return kartsnitt

def geokart_distinct(steder, limit=50):
    """ dict of places: {place: [lat, lng], ...} """
    import numpy as np
    
    #lat, lon = koord(sted)
    kartsnitt = folium.Map(location=[60, 10], zoom_start = 6)
    for item in steder:
        kartsnitt.add_child(MarkerCluster([steder[item]], popups=[item]))
    #print(locs)
    return kartsnitt

def geokart_circles(steder, weights, name = 'alle', color ='#2244AA', radius = 100000, thickness = 10):
    """ dict of places: {place: [lat, lng], ...} """
    import numpy as np
    group = folium.FeatureGroup(name=name)
    #lat, lon = koord(sted)
    kartsnitt = folium.Map(location=[60, 10], zoom_start = 6)
    for item in steder:
        if item in weights:
            folium.Circle(location=steder[item], radius = weights[item]*radius,
                    popup = item,
                    color = color, weight= thickness, fill = False).add_to(group)
    kartsnitt.add_child(group)
    #folium.LayerControl().add_to(kartsnitt)
    #print(locs)
    return kartsnitt

def add_layer(steder, kart,  weights, name, color = 'red', thickness = 10, radius=100000):
    group = folium.FeatureGroup(name=name)
    for item in steder:
        folium.Circle(location=steder[item], radius = weights[item]*radius,
                    popup = item, line_color = color,
                    color = color, fill = False, weight=thickness).add_to(group)
    group.add_to(kart)
    #folium.LayerControl().add_to(kart)
    return kart

def geokart_add(kartsnitt, sted):
    lat, lon = koord(sted)
    #kartsnitt = folium.Map(location=[lat, lon], zoom_start = 6)
    locs = []
    pops = []
    #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
    kartsnitt.add_child(MarkerCluster([[lat,lon]], popups=[sted]))
    #print(locs)
    return kartsnitt



# Henter inn navnene

# Lager et kart over alle stedsnavn

In [51]:
nb.xmlpretty(finnsted('Egypten'))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<html>
 <body>
  <sokres>
   <sokstatus>
    <ok>
     true
    </ok>
    <melding>
    </melding>
   </sokstatus>
   <totaltantalltreff>
    0
   </totaltantalltreff>
  </sokres>
 </body>
</html>


In [5]:
def geo_info(n, pi):
    d = pi['components']
    d.update(pi['geometry'])
    d['_name'] = n
    return d

In [6]:
hamsun_all = {'Aabakken': 20.0,
 'Aas': 128.0,
 'Afrika': 31.0,
 'Amerika': 1104.0,
 'Ananur': 7.0,
 'Argentina': 21.0,
 'Armenia': 7.0,
 'Asien': 22.0,
 'Asker': 29.0,
 'Atlanta': 3.0,
 'Atlanterhavet': 27.0,
 'Aurdal': 155.0,
 'Australien': 33.0,
 'Baku': 40.0,
 'Balberg': 18.0,
 'Baltimore': 1.0,
 'Batum': 6.0,
 'Bayern': 5.0,
 'Belgien': 40.0,
 'Bergen': 426.0,
 'Berlin': 227.0,
 'Bjærget': 38.0,
 'Bodø': 102.0,
 'Bokhara': 15.0,
 'Bolivia': 9.0,
 'Boston': 26.0,
 'Brasil': 2.0,
 'Breidablik': 48.0,
 'Brevik': 6.0,
 'Bryne': 2.0,
 'Byen': 1633.0,
 'Bygdø': 28.0,
 'California': 5.0,
 'Chicago': 72.0,
 'Christiania': 2029.0,
 'Cordilleras': 4.0,
 'Dakota': 19.0,
 'Danmark': 250.0,
 'Danzig': 5.0,
 'Doppen': 209.0,
 'Drammen': 40.0,
 'Drøbak': 82.0,
 'Dubrovnik': 20.0,
 'Egersund': 18.0,
 'Egypten': 23.0,
 'Elverum': 24.0,
 'England': 404.0,
 'Equador': 1.0,
 'Europa': 213.0,
 'Finland': 138.0,
 'Finmarken': 40.0,
 'Fjell': 20.0,
 'Flandern': 2.0,
 'Florø': 4.0,
 'Fosenlandet': 40.0,
 'Frankrike': 143.0,
 'Fredrikstad': 9.0,
 'Fuglværø': 14.0,
 'Gaarden': 713.0,
 'Gaden': 538.0,
 'Ganges': 5.0,
 'Garmo': 17.0,
 'Gaustad': 9.0,
 'Gjørvik': 10.0,
 'Gjøvik': 52.0,
 'Grand_Hotel': 136.0,
 'Grekenland': 19.0,
 'Grimstad': 716.0,
 'Grønland': 67.0,
 'Gudbrandsdalen': 21.0,
 'Guinea': 4.0,
 'Haag': 8.0,
 'Hamar': 17.0,
 'Hamarøy': 346.0,
 'Hammerfest': 5.0,
 'Hamsund': 73.0,
 'Hannover': 55.0,
 'Hardanger': 34.0,
 'Harrow': 12.0,
 'Harstad': 32.0,
 'Haugesund': 12.0,
 'Havet': 213.0,
 'Heia': 23.0,
 'Helsingfors': 117.0,
 'Holmestrand': 10.0,
 'Hornbæk': 25.0,
 'Horten': 12.0,
 'Hønefoss': 5.0,
 'Hørup': 15.0,
 'Indien': 67.0,
 'Indrebygden': 62.0,
 'Italien': 43.0,
 'Japan': 28.0,
 'Jerusalem': 28.0,
 'Jorden': 714.0,
 'Judea': 3.0,
 'Kairo': 1.0,
 'Kanada': 49.0,
 'Kansas': 4.0,
 'Kaukasien': 46.0,
 'Kentucky': 26.0,
 'Kina': 19.0,
 'Kongsberg': 69.0,
 'Konstantinopel': 41.0,
 'Kreml': 5.0,
 'Kristianssand': 52.0,
 'Kristiansund': 66.0,
 'Kvæfjord': 6.0,
 'Kystby': 6.0,
 'København': 621.0,
 'Landvik': 199.0,
 'Larvik': 206.0,
 'Levanger': 16.0,
 'Lillehammer': 33.0,
 'Lillesand': 245.0,
 'Lofoten': 154.0,
 'Lom': 38.0,
 'London': 79.0,
 'Lyon': 6.0,
 'Maaneland': 32.0,
 'Mandal': 4.0,
 'Marken': 531.0,
 'Marseille': 3.0,
 'Mexico': 31.0,
 'Midtvesten': 1.0,
 'Minneapolis': 100.0,
 'Mleti': 6.0,
 'Molde': 11.0,
 'Montana': 3.0,
 'Moskva': 69.0,
 'Munchen': 30.0,
 'Myren': 193.0,
 'Narvik': 20.0,
 'Natal': 14.0,
 'Nationen': 74.0,
 'Nevada': 5.0,
 'New_Orleans': 9.0,
 'New_Zealand': 8.0,
 'Niam-Niam': 1.0,
 'Norden': 101.0,
 'Nordland': 193.0,
 'Norge': 1168.0,
 'Nuten': 15.0,
 'Nørholmen': 1280.0,
 'Odessa': 4.0,
 'Ohio': 10.0,
 'Orienten': 20.0,
 'Ostende': 15.0,
 'Otta': 2.0,
 'Paris': 432.0,
 'Pennsylvania': 1.0,
 'Persien': 23.0,
 'Peru': 4.0,
 'Petersburg': 19.0,
 'Polden': 524.0,
 'Polen': 29.0,
 'Portugal': 6.0,
 'Prærien': 5.0,
 'Quebec': 4.0,
 'Roma': 49.0,
 'Rosengård': 20.0,
 'Rostov': 1.0,
 'Rusland': 202.0,
 'Saba': 41.0,
 'Sagvika': 30.0,
 'Samsø': 36.0,
 'Sarpsborg': 31.0,
 'Segelfoss': 530.0,
 'Sellanraa': 139.0,
 'Serbien': 12.0,
 'Sibir': 9.0,
 'Sirilund': 212.0,
 'Skandinavien': 29.0,
 'Skogen': 609.0,
 'Småbyen': 2.0,
 'Sortehavet': 16.0,
 'Spania': 27.0,
 'Spitsbergen': 5.0,
 'Stavanger': 48.0,
 'Steppen': 21.0,
 'Stockholm': 162.0,
 'Stokmarknes': 20.0,
 'Storborg': 42.0,
 'Stordalen': 13.0,
 'Stortinget': 124.0,
 'Stranden': 3.0,
 'Suezkanalen': 5.0,
 'Sveits': 57.0,
 'Sverige': 217.0,
 'Svolvær': 9.0,
 'Sydafrika': 3.0,
 'Sydamerika': 48.0,
 'Syden': 33.0,
 'Sørlandet': 13.0,
 'Theresegade': 38.0,
 'Tiflis': 58.0,
 'Torahus': 80.0,
 'Toretind': 24.0,
 'Torpelviken': 31.0,
 'Tromsø': 60.0,
 'Trondhjem': 171.0,
 'Tvedestrand': 3.0,
 'Tyrkia': 52.0,
 'Tyskland': 685.0,
 'Ungarn': 12.0,
 'Ura': 23.0,
 'Utvær': 22.0,
 'Vadsø': 5.0,
 'Valdres': 126.0,
 'Vangen': 29.0,
 'Venezia': 2.0,
 'Verden': 1348.0,
 'Vesteraalen': 19.0,
 'Vestfjorden': 17.0,
 'Vladikaukas': 26.0,
 'Vælta': 17.0,
 'Washington': 30.0,
 'Wien': 71.0,
 'Wisconsin': 42.0,
 'Ytrepolden': 17.0,
 'Ødemarken': 22.0,
 'Østen': 71.0}

In [7]:
hamsun_brev = {'Aas': 118.0,
 'Afrika': 8.0,
 'Amerika': 341.0,
 'Argentina': 2.0,
 'Asien': 5.0,
 'Asker': 29.0,
 'Atlanta': 1.0,
 'Atlanterhavet': 11.0,
 'Aurdal': 147.0,
 'Australien': 6.0,
 'Baku': 6.0,
 'Balberg': 18.0,
 'Batum': 1.0,
 'Bayern': 4.0,
 'Belgien': 17.0,
 'Bergen': 259.0,
 'Berlin': 204.0,
 'Bodø': 61.0,
 'Boston': 6.0,
 'Brasil': 2.0,
 'Brevik': 5.0,
 'Byen': 162.0,
 'Bygdø': 27.0,
 'California': 3.0,
 'Chicago': 38.0,
 'Christiania': 1812.0,
 'Dakota': 6.0,
 'Danmark': 169.0,
 'Drammen': 26.0,
 'Drøbak': 82.0,
 'Dubrovnik': 19.0,
 'Egersund': 18.0,
 'Egypten': 4.0,
 'Elverum': 21.0,
 'England': 101.0,
 'Europa': 50.0,
 'Finland': 79.0,
 'Finmarken': 7.0,
 'Fjell': 16.0,
 'Frankrike': 83.0,
 'Fredrikstad': 9.0,
 'Gaarden': 159.0,
 'Gaden': 56.0,
 'Garmo': 16.0,
 'Gaustad': 2.0,
 'Gjørvik': 6.0,
 'Gjøvik': 48.0,
 'Grand_Hotel': 58.0,
 'Grekenland': 1.0,
 'Grimstad': 682.0,
 'Grønland': 26.0,
 'Gudbrandsdalen': 13.0,
 'Haag': 7.0,
 'Hamar': 16.0,
 'Hamarøy': 327.0,
 'Hammerfest': 2.0,
 'Hamsund': 72.0,
 'Hannover': 27.0,
 'Hardanger': 20.0,
 'Harstad': 30.0,
 'Haugesund': 7.0,
 'Havet': 27.0,
 'Helsingfors': 105.0,
 'Holmestrand': 6.0,
 'Hornbæk': 24.0,
 'Horten': 11.0,
 'Hønefoss': 4.0,
 'Hørup': 15.0,
 'Indien': 14.0,
 'Italien': 17.0,
 'Japan': 18.0,
 'Jerusalem': 4.0,
 'Jorden': 75.0,
 'Judea': 2.0,
 'Kanada': 11.0,
 'Kaukasien': 14.0,
 'Kina': 1.0,
 'Kongsberg': 67.0,
 'Konstantinopel': 16.0,
 'Kreml': 1.0,
 'Kristianssand': 47.0,
 'Kristiansund': 63.0,
 'København': 556.0,
 'Landvik': 192.0,
 'Larvik': 194.0,
 'Levanger': 1.0,
 'Lillehammer': 30.0,
 'Lillesand': 232.0,
 'Lofoten': 4.0,
 'Lom': 28.0,
 'London': 33.0,
 'Lyon': 6.0,
 'Mandal': 3.0,
 'Marken': 131.0,
 'Marseille': 1.0,
 'Midtvesten': 1.0,
 'Minneapolis': 74.0,
 'Molde': 8.0,
 'Moskva': 42.0,
 'Munchen': 23.0,
 'Myren': 29.0,
 'Narvik': 19.0,
 'Nationen': 53.0,
 'New_Orleans': 3.0,
 'New_Zealand': 5.0,
 'Norden': 50.0,
 'Nordland': 103.0,
 'Norge': 738.0,
 'Nørholmen': 1247.0,
 'Odessa': 3.0,
 'Ohio': 2.0,
 'Orienten': 6.0,
 'Ostende': 14.0,
 'Otta': 1.0,
 'Paris': 359.0,
 'Persien': 1.0,
 'Petersburg': 14.0,
 'Polden': 1.0,
 'Polen': 13.0,
 'Portugal': 4.0,
 'Roma': 24.0,
 'Rusland': 95.0,
 'Saba': 20.0,
 'Samsø': 36.0,
 'Sarpsborg': 29.0,
 'Segelfoss': 109.0,
 'Sellanraa': 1.0,
 'Serbien': 5.0,
 'Sibir': 2.0,
 'Sirilund': 8.0,
 'Skandinavien': 15.0,
 'Skogen': 42.0,
 'Sortehavet': 2.0,
 'Spania': 9.0,
 'Spitsbergen': 1.0,
 'Stavanger': 33.0,
 'Stockholm': 140.0,
 'Stokmarknes': 2.0,
 'Stortinget': 42.0,
 'Sveits': 20.0,
 'Sverige': 122.0,
 'Svolvær': 8.0,
 'Sydafrika': 2.0,
 'Sydamerika': 11.0,
 'Syden': 5.0,
 'Sørlandet': 11.0,
 'Theresegade': 38.0,
 'Tiflis': 4.0,
 'Tromsø': 21.0,
 'Trondhjem': 31.0,
 'Tvedestrand': 2.0,
 'Tyrkia': 13.0,
 'Tyskland': 487.0,
 'Ungarn': 5.0,
 'Vadsø': 1.0,
 'Valdres': 115.0,
 'Venezia': 2.0,
 'Verden': 342.0,
 'Vesteraalen': 7.0,
 'Vladikaukas': 1.0,
 'Washington': 4.0,
 'Wien': 61.0,
 'Wisconsin': 33.0,
 'Ødemarken': 1.0,
 'Østen': 13.0}

In [8]:
hamsun_fiksjon = {'Aabakken': 20.0,
 'Aas': 10.0,
 'Afrika': 16.0,
 'Amerika': 270.0,
 'Ananur': 7.0,
 'Argentina': 19.0,
 'Armenia': 7.0,
 'Asien': 2.0,
 'Atlanterhavet': 7.0,
 'Aurdal': 1.0,
 'Australien': 26.0,
 'Baku': 34.0,
 'Batum': 5.0,
 'Belgien': 1.0,
 'Bergen': 126.0,
 'Berlin': 18.0,
 'Bjærget': 38.0,
 'Bodø': 39.0,
 'Bokhara': 15.0,
 'Bolivia': 9.0,
 'Boston': 1.0,
 'Breidablik': 48.0,
 'Brevik': 1.0,
 'Bryne': 2.0,
 'Byen': 1419.0,
 'California': 1.0,
 'Chicago': 8.0,
 'Christiania': 155.0,
 'Cordilleras': 4.0,
 'Dakota': 3.0,
 'Danmark': 5.0,
 'Doppen': 209.0,
 'Drammen': 10.0,
 'Egypten': 3.0,
 'Elverum': 2.0,
 'England': 97.0,
 'Equador': 1.0,
 'Europa': 40.0,
 'Finland': 35.0,
 'Finmarken': 31.0,
 'Florø': 4.0,
 'Fosenlandet': 38.0,
 'Frankrike': 20.0,
 'Fuglværø': 14.0,
 'Gaarden': 553.0,
 'Gaden': 443.0,
 'Ganges': 2.0,
 'Garmo': 1.0,
 'Gaustad': 3.0,
 'Gjøvik': 1.0,
 'Grand_Hotel': 67.0,
 'Grekenland': 13.0,
 'Grimstad': 14.0,
 'Grønland': 2.0,
 'Gudbrandsdalen': 3.0,
 'Guinea': 3.0,
 'Hamar': 1.0,
 'Hamarøy': 5.0,
 'Hammerfest': 2.0,
 'Hannover': 27.0,
 'Hardanger': 9.0,
 'Harrow': 12.0,
 'Haugesund': 3.0,
 'Havet': 167.0,
 'Heia': 23.0,
 'Helsingfors': 8.0,
 'Holmestrand': 3.0,
 'Horten': 1.0,
 'Hønefoss': 1.0,
 'Indien': 39.0,
 'Indrebygden': 62.0,
 'Italien': 7.0,
 'Japan': 9.0,
 'Jerusalem': 21.0,
 'Jorden': 600.0,
 'Kairo': 1.0,
 'Kanada': 18.0,
 'Kaukasien': 32.0,
 'Kentucky': 23.0,
 'Kina': 12.0,
 'Kongsberg': 1.0,
 'Konstantinopel': 25.0,
 'Kreml': 4.0,
 'Kristianssand': 5.0,
 'Kvæfjord': 6.0,
 'Kystby': 6.0,
 'København': 15.0,
 'Landvik': 5.0,
 'Levanger': 15.0,
 'Lillesand': 1.0,
 'Lofoten': 149.0,
 'Lom': 5.0,
 'London': 32.0,
 'Maaneland': 32.0,
 'Mandal': 1.0,
 'Marken': 387.0,
 'Marseille': 1.0,
 'Mexico': 27.0,
 'Mleti': 6.0,
 'Montana': 1.0,
 'Moskva': 26.0,
 'Munchen': 3.0,
 'Myren': 164.0,
 'Natal': 14.0,
 'Nationen': 3.0,
 'New_Orleans': 4.0,
 'New_Zealand': 1.0,
 'Norden': 3.0,
 'Nordland': 51.0,
 'Norge': 154.0,
 'Nuten': 15.0,
 'Nørholmen': 10.0,
 'Odessa': 1.0,
 'Orienten': 10.0,
 'Otta': 1.0,
 'Paris': 43.0,
 'Persien': 19.0,
 'Peru': 3.0,
 'Petersburg': 4.0,
 'Polden': 523.0,
 'Polen': 7.0,
 'Prærien': 2.0,
 'Quebec': 3.0,
 'Roma': 7.0,
 'Rosengård': 20.0,
 'Rostov': 1.0,
 'Rusland': 88.0,
 'Saba': 17.0,
 'Sagvika': 30.0,
 'Segelfoss': 418.0,
 'Sellanraa': 137.0,
 'Serbien': 6.0,
 'Sirilund': 204.0,
 'Skogen': 564.0,
 'Sortehavet': 14.0,
 'Spania': 12.0,
 'Spitsbergen': 2.0,
 'Steppen': 21.0,
 'Stockholm': 13.0,
 'Stokmarknes': 17.0,
 'Storborg': 42.0,
 'Stordalen': 13.0,
 'Stortinget': 47.0,
 'Stranden': 3.0,
 'Suezkanalen': 1.0,
 'Sveits': 23.0,
 'Sverige': 51.0,
 'Svolvær': 1.0,
 'Sydafrika': 1.0,
 'Sydamerika': 37.0,
 'Syden': 4.0,
 'Sørlandet': 2.0,
 'Tiflis': 54.0,
 'Torahus': 80.0,
 'Toretind': 24.0,
 'Torpelviken': 31.0,
 'Tromsø': 22.0,
 'Trondhjem': 133.0,
 'Tvedestrand': 1.0,
 'Tyrkia': 36.0,
 'Tyskland': 25.0,
 'Ungarn': 5.0,
 'Ura': 23.0,
 'Utvær': 22.0,
 'Vadsø': 4.0,
 'Valdres': 3.0,
 'Vangen': 29.0,
 'Verden': 908.0,
 'Vesteraalen': 12.0,
 'Vestfjorden': 14.0,
 'Vladikaukas': 24.0,
 'Vælta': 17.0,
 'Wien': 6.0,
 'Wisconsin': 3.0,
 'Ytrepolden': 17.0,
 'Ødemarken': 21.0,
 'Østen': 41.0}

In [9]:
navnematrise = nb.frame({'alle':hamsun_all, 'brev':hamsun_brev, 'fiksjon':hamsun_fiksjon}).transpose().fillna(0)

In [10]:
navnematrise

,alle,fiksjon,brev
Aabakken,20.0,20.0,0.0
Aas,128.0,10.0,118.0
Afrika,31.0,16.0,8.0
Amerika,1104.0,270.0,341.0
Ananur,7.0,7.0,0.0
...,...,...,...
Wien,71.0,6.0,61.0
Wisconsin,42.0,3.0,33.0
Ytrepolden,17.0,17.0,0.0
Ødemarken,22.0,21.0,1.0


# gjør om til kode for å kjøre på nytt
m = []
for n in navn.index:
    m += [geo_info(n, pi) for pi in place_info(n)]

In [11]:
import dhlab.token_map as tm

In [12]:
#df = pd.DataFrame(m)
df = pd.read_csv('hamsun_geo_tagged.csv')

In [24]:
no = pd.read_csv('NO/NO.txt', sep='\t', header=None)

C:\Users\larsj\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
no = no[[1,2,3,4,5]]

In [17]:
print('\n'.join(df.columns))

Unnamed: 0
ISO_3166-1_alpha-2
ISO_3166-1_alpha-3
_name
_type
administrative
aerodrome
allotments
archaeological_site
arts_centre
artwork
attraction
bakery
bar
bay
beach
building
bus_stop
cafe
canal
car_rental
chalet
city
city_district
clothes
commercial
community_centre
continent
convenience
country
country_code
county
county_code
cycleway
department_store
doctors
dry_cleaning
farm
farmyard
fast_food
ferry_terminal
florist
footway
fuel
garden
grave_yard
guest_house
hamlet
hardware
hostel
hotel
house
house_number
industrial
information
island
islet
lat
lng
local_administrative_area
locality
marina
memorial
mountain_pass
neighbourhood
path
peak
pedestrian
picnic_site
place_of_worship
political_union
postcode
pub
raceway
railway
recreation_ground
region
residential
restaurant
retail
river
road
road_type
sports_centre
stadium
state
state_code
state_district
station
stream
suburb
subway_entrance
supermarket
taxi
town
toys
travel_agency
university
unknown
viewpoint
village
water


In [27]:
no

,1,2,3,4,5
0,Jakobselvvatnet,Jakobselvvatnet,"Jakobselvvand,Jakobselvvann,Jakobselvvatn,Jako...",69.55117,30.91213
1,Vaggatem,Vaggatem,"Ozero Bukhtles-Vandet,Ozero Vouvatusyarvi,Ozer...",69.27159,29.26756
2,Jakobselva,Jakobselva,"Jakobselv,Jakobselva,Vor'yema,Vor’yema,Vuoremi...",69.77440,30.83950
3,Porog Vakker-Striket,Porog Vakker-Striket,"Gaddoluobalguoikka,Porog Vakker-Striket,Vaggat...",69.30070,29.31574
4,Tangenfossen,Tangenfossen,"Bornjeguoikka,Buornesava,Porog Tange-Foss,Purn...",69.07358,29.17126
...,...,...,...,...,...
605832,Nesbyen,Nesbyen,NaN,60.55142,9.07253
605833,Alver,Alver,NaN,60.67188,5.28350
605834,Hustadvika,Hustadvika,NaN,62.88626,7.22540
605835,Gulli Gård,Gulli Gard,NaN,59.62131,10.85401


In [57]:
print(', '.join(no[no[3].str.contains('Berga')].iloc[0].split(',')))

ValueError: cannot index with vector containing NA / NaN values

In [19]:
df

,Unnamed: 0,ISO_3166-1_alpha-2,ISO_3166-1_alpha-3,_name,_type,administrative,aerodrome,allotments,archaeological_site,arts_centre,...,supermarket,taxi,town,toys,travel_agency,university,unknown,viewpoint,village,water
0,0,NO,NOR,Arendal,county,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NO,NOR,Arendal,city,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Arendal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,SE,SWE,Arendal,neighbourhood,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,SE,SWE,Arendal,village,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NO,NOR,Arendal,farm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544,1544,AT,AUT,Ganges,peak,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1545,1545,US,USA,Ganges,village,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1546,1546,US,USA,Ganges,village,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ganges,NaN
1547,1547,NO,NOR,Kvæfjord,county,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
hw = list(set(df["_name"]))

In [18]:
hw

['Myren',
 'Konstantinopel',
 'Tyskland',
 'Balberg',
 'Prærien',
 'California',
 'Nationen',
 'Marken',
 'Sveits',
 'Egersund',
 'Hammerfest',
 'Serbien',
 'Norden',
 'Kristiansund',
 'Afrika',
 'Saba',
 'Helsingfors',
 'Aabakken',
 'Tiflis',
 'Kentucky',
 'Nordland',
 'Trondhjem',
 'Svolvær',
 'Wien',
 'Brasil',
 'Rosengård',
 'Skandinavien',
 'Tromsø',
 'Moskva',
 'Mandal',
 'Dakota',
 'Indien',
 'Quebec',
 'Fredrikstad',
 'Danmark',
 'Europa',
 'New York',
 'Marseille',
 'Sydafrika',
 'Heia',
 'Berg Sanatorium',
 'Sellanraa',
 'Australien',
 'Grimstad',
 'Jorden',
 'Ananur',
 'Nørholmen',
 'Sirilund',
 'Steppen',
 'Molde',
 'København',
 'Munchen',
 'Syden',
 'Aas',
 'Vælta',
 'London',
 'Berlin',
 'Utvær',
 'Frankrike',
 'Kansas',
 'Stranden',
 'Rådstuen',
 'Stokmarknes',
 'Haugesund',
 'Havet',
 'Valdres',
 'Natal',
 'Gjøvik',
 'Hannover',
 'Atlanterhavet',
 'Ungarn',
 'Ohio',
 'Bodø',
 'Judea',
 'Sydamerika',
 'Danzig',
 'Sagvika',
 'Horten',
 'Mleti',
 'Christiania',
 'Ostende'

In [19]:
import dhlab.nbtext as nb

In [24]:
df = df[["_name", "country", "postcode", "_type", "lat", "lng"]]

In [81]:
df['in']

''

In [31]:
edits = qg.show_grid(df, show_toolbar = True)
edits

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [32]:
steder_disambig = edits.get_changed_df()
steder_disambig

,_name,country,postcode,_type,lat,lng,in
1,Arendal,Norway,4836,city,58.459641,8.766602,1
7,Argentina,Argentina,NaN,country,-34.996496,-64.967282,1
17,Asker,Norway,NaN,county,59.838578,10.457773,1
26,Aurdal,Norway,2910,village,60.924233,9.415942,1
30,Baku,Azerbaijan,NaN,city,40.375443,49.832675,1
...,...,...,...,...,...,...,...
1504,Rostov,Russia,NaN,city,57.186013,39.424956,1
1512,Sibir,Russia,NaN,county,60.000000,100.000000,1
1522,Odessa,Ukraine,65026,city,46.487319,30.739278,1
1537,Ganges,India,249304,river,25.605269,83.703847,1


In [21]:
#pd.DataFrame(m).to_csv('hamsun_geo_tagged.csv')

In [33]:
steder_disambig.to_csv('disambiguerte_steder_hamsun.csv')

In [22]:
from scipy.special import softmax

In [23]:
#hamsun_names.to_csv('hamsun_steder_frekvens.csv')

# Se på kartet

Her kan vi gjøre betydelig mer, men selve geotaggingen går greit. 

In [34]:
stedsinfo = steder_disambig[['_name', 'lat','lng', 'country']]

In [35]:
steder = dict()
for _, row in stedsinfo.iterrows():
    steder[row['_name']] = [row['lat'], row['lng']]
steder

{'Arendal': [58.459640799999995, 8.7666023],
 'Argentina': [-34.9964963, -64.9672817],
 'Asker': [59.838578399999996, 10.4577725],
 'Aurdal': [60.924232700000005, 9.4159418],
 'Baku': [40.3754434, 49.8326748],
 'Balberg': [61.1556321, 10.422768300000001],
 'Bergen': [60.39430550000001, 5.3259192],
 'Bjærget': [57.10547, 8.99315],
 'Bodø': [67.28394759999999, 14.3825834],
 'Bokhara': [26.3588905, 85.6512634],
 'Boston': [42.3602534, -71.0582912],
 'Breidablik': [60.6110083, 6.4209860999999995],
 'Bygdø': [59.90715229999999, 10.6796457],
 'Chicago': [41.875561600000005, -87.6244212],
 'Christiania': [59.913330099999996, 10.7389701],
 'Danmark': [55.670249, 10.333328300000002],
 'Drammen': [59.743674899999995, 10.204374900000001],
 'Drøbak': [59.6633333, 10.631111099999998],
 'Dubrovnik': [42.6499651, 18.093652],
 'Egersund': [58.4518952, 5.999996299999999],
 'Elverum': [60.88296329999999, 11.560238400000001],
 'England': [52.795479099999994, -0.5402403],
 'Equador': [-1.3397668, -79.3666

In [42]:
brev = {x:steder[x] for x in hamsun_brev if x in steder}

In [43]:
fiksjon = {x:steder[x] for x in hamsun_fiksjon if x in steder}

In [40]:
def normalise_json(struct):
    tot = sum(struct.values())
    for x in struct:
        struct[x] = struct[x]/tot
    return struct

In [85]:
norske_brev

{'Aas': [58.331738, 8.276099],
 'Afrika': [58.1831647, 8.1458053],
 'Asker': [59.8330064, 10.4339954],
 'Atlanterhavet': [58.688559999999995, 7.12868],
 'Aurdal': [59.55935899999999, 5.722217],
 'Balberg': [60.683001000000004, 11.1688258],
 'Belgien': [59.405911100000004, 8.4309222],
 'Bergen': [60.39430550000001, 5.3259192],
 'Bodø': [67.3094783, 13.915442],
 'Brevik': [59.5027241, 10.6890562],
 'Byen': [58.21705, 7.83476],
 'Bygdø': [59.90715229999999, 10.6796457],
 'Christiania': [59.913330099999996, 10.7389701],
 'Danmark': [59.0525929, 9.9212469],
 'Drammen': [57.995838899999995, 7.4885],
 'Drøbak': [59.660984400000004, 10.627730999999999],
 'Egersund': [58.461351, 6.0023805],
 'Elverum': [69.02123990000001, 18.803774899999997],
 'England': [59.5201583, 8.194616700000001],
 'Finland': [59.8125924, 11.111375500000001],
 'Finmarken': [59.9, 10.08333],
 'Fjell': [59.036513899999996, 11.4406444],
 'Frankrike': [58.961951, 9.558561],
 'Fredrikstad': [59.208898100000006, 10.950579800000

In [73]:
verden = geokart_circles(steder, normalise_json(hamsun_all), thickness=6, radius=300000)
verden

In [74]:
verden.save('hamsuns_verden.html')

In [75]:
layered = add_layer(brev, verden, normalise_json(hamsun_brev), 'brev', color='red', radius=100000, thickness = 6)
all_layers = add_layer(fiksjon, layered, normalise_json(hamsun_fiksjon), 'fiksjon', color='green', radius=100000, thickness=6)
folium.LayerControl().add_to(all_layers)

In [76]:
layered

In [77]:
layered.save('hamsun_brev_fiksjon.html')

In [134]:
all_layers = add_layer(norske_fiksjon, layered, normalise_json(hamsun_fiksjon), name='fiksjon', color='yellow')
folium.LayerControl().add_to(all_layers)

In [135]:
all_layers

In [136]:
all_layers.save('hamsun_steder_lagvis.html')

In [30]:
geokart_circles(norske).save('circles.html')

In [116]:
krt.save('kart.html')

In [31]:
nor = pgeo.Nominatim('no')

In [32]:
nor.query_location("oslo")

In [33]:
nor.query_postal_code("0271")

postal_code              0271
country code               NO
place_name               Oslo
state_name        Oslo County
state_code                 12
county_name              Oslo
county_code               301
community_name            NaN
community_code            NaN
latitude              59.8457
longitude             10.7823
accuracy                  NaN
Name: 0, dtype: object

In [34]:
opencage_api = "a8e2de88b5b64323a9b7668a0ee6e7b6"

In [35]:
koord("trondheim")

(63.4305658, 10.3951929)

In [36]:
res = place_info("Hammerfest")
len(res)

3

In [37]:
[(r['components'], r['geometry']) for r in res]

[({'ISO_3166-1_alpha-2': 'NO',
   'ISO_3166-1_alpha-3': 'NOR',
   '_type': 'county',
   'continent': 'Europe',
   'country': 'Norway',
   'country_code': 'no',
   'county': 'Finnmark'},
  {'lat': 70.6413392, 'lng': 23.8360801}),
 ({'ISO_3166-1_alpha-2': 'NO',
   'ISO_3166-1_alpha-3': 'NOR',
   '_type': 'city',
   'continent': 'Europe',
   'country': 'Norway',
   'country_code': 'no',
   'county': 'Finnmark',
   'postcode': '9600',
   'town': 'Hammerfest'},
  {'lat': 70.6636329, 'lng': 23.6791211}),
 ({'ISO_3166-1_alpha-2': 'IT',
   'ISO_3166-1_alpha-3': 'ITA',
   '_type': 'road',
   'continent': 'Europe',
   'country': 'Italy',
   'country_code': 'it',
   'county': 'Comunità Valsugana e Tesino',
   'neighbourhood': 'Maso Vazzena',
   'political_union': 'European Union',
   'postcode': '38050',
   'road': 'Strada Statale 47 della Valsugana',
   'state': 'Trentino-Alto Adige/Südtirol',
   'suburb': 'Larganza',
   'unknown': 'Hammerfest',
   'village': 'Roncegno Terme'},
  {'lat': 46.0417

In [38]:
results[0]['geometry']

NameError: name 'results' is not defined

In [39]:
results[0]["components"]["_type"]

NameError: name 'results' is not defined

In [40]:
import pandas as pd

In [41]:
pd.DataFrame(results)

NameError: name 'results' is not defined

In [42]:
import utm
utm.to_latlon(340000, 5710000, 32, 'U')
(51.51852098408468, 6.693872395145327)

ModuleNotFoundError: No module named 'utm'

In [43]:
from convertbng.util import convert_bng, convert_lonlat,convert_etrs89_to_lonlat,convert_epsg3857_to_wgs84


ModuleNotFoundError: No module named 'convertbng'

In [44]:
for x in koord('Tromsø', 'By'):
    e = float(x[1])
    n = float(x[2])
    print(e, n)
    (a, b) = (utm.to_latlon(e, n, 32,  'u'))
    print((a,b))


TypeError: koord() takes 1 positional argument but 2 were given